### Import modules

In [ ]:
from scrapper.extract_artcls_data import current_artcls, artcl_data, artcl_content
from scrapper.location_detector import detect_location, validate_location, get_location_id
from scrapper.art_hash import get_hash
from database_manager.database_connector import dbconnection as db
from database_manager import querys


### Save url containing articles

In [ ]:
# Base url
news_url= "https://www.elnuevodia.com/noticias/seguridad"

### Construct data dictionary and insert data into database

In [ ]:
db.reconnect()
cursor = db.cursor(buffered=True)

# Fetch stored articles hashes
cursor.execute(querys.FETCH_HASHES)

# Create list of hashes
hashes = []
for row in cursor:
    hashes.append(row[0])

# Get current articles
articles = current_artcls(news_url)

# Iterate over current articles
for article in articles:
    # Get article has
    art_hash = get_hash(article=article)
    # Get article data and store to database only of hash is not in database already
    if art_hash not in hashes:

        data_dict = artcl_data(article=article)
        
        data_dict.update(artcl_content(article_data=data_dict))
        
        # Execute insert and commit to database
        cursor.execute(querys.INSERT_ARTICLE, data_dict)
        db.commit()

        # Get ID of inserted article
        art_id = cursor.lastrowid

        locs_found = detect_location(article_data=data_dict)

        loc_validated = validate_location(location=locs_found)
        
        if loc_validated:
            for loc in loc_validated:
                
                loc_id = get_location_id(validated_location=loc)

                # To do: Insert (ArticleID, LocationID) into ArticleLocationRelationship table
                # using the article ID fetched from database after inserted, and loc_id in this
                # for loop
                data_dict = {"articleid": art_id, "locationid": loc_id}
                cursor.execute(querys.INSERT_ARTICLE_LOCATION, data_dict)
                db.commit()

### Print Articles table rows

In [ ]:
query = """SELECT * FROM Articles"""
cursor.execute(query)
for row in cursor:
    print(row)

## Print Location rows

In [ ]:
query = """SELECT * FROM Location"""
cursor.execute(query)
for row in cursor:
    print(row)

## Print bridging table rows

In [ ]:
query = """SELECT * FROM ArticlesLocationRelation"""
cursor.execute(query)
for row in cursor:
    print(row)

## Close database connection

In [ ]:
db.close()